In [537]:
import pandas as pd
import seaborn as sns
import requests
from bs4 import BeautifulSoup
import category_encoders as ce
from geopy.distance import geodesic
from geopy import distance

In [538]:
pd.set_option('display.max_columns', None)

In [539]:
# Загрузив датасет, видим пропуски в двух столбцах: с широтой и долготой расположения отеля. 

df = pd.read_csv('hotels.csv')
df.head()
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 386803 entries, 0 to 386802
Data columns (total 17 columns):
 #   Column                                      Non-Null Count   Dtype  
---  ------                                      --------------   -----  
 0   hotel_address                               386803 non-null  object 
 1   additional_number_of_scoring                386803 non-null  int64  
 2   review_date                                 386803 non-null  object 
 3   average_score                               386803 non-null  float64
 4   hotel_name                                  386803 non-null  object 
 5   reviewer_nationality                        386803 non-null  object 
 6   negative_review                             386803 non-null  object 
 7   review_total_negative_word_counts           386803 non-null  int64  
 8   total_number_of_reviews                     386803 non-null  int64  
 9   positive_review                             386803 non-null  object 
 

In [540]:
# Удалим эти строки, так как данные признаки я буду использовать для создания нового - расстояния отеля от центра города.

drop_list = df[(df['lat'].isnull() == True) | (df['lng'].isnull() == True) ].index.tolist()
len(drop_list)

df = df.drop(drop_list, axis = 0)

In [541]:
# Посмотрим на категориальные и числовые признаки

columns = df.columns.to_list()
num_cols = list()
cat_cols = list()

for column in columns:
    if df[column].dtype == 'O':
        cat_cols.append(column)
    else:
        num_cols.append(column)

len(cat_cols)
len(num_cols)

9

In [542]:
# В DataFrame 'feature_df' буду вносить числовые признаки, а так же обработанные категориальные, 
# чтобы использовать эти данные для обучения модели.

feature_df = pd.DataFrame()

# Преобразования для ответа на задания на учебной площадке:

df['hotel_name'].nunique()
feature_df['review_date'] = pd.to_datetime(df['review_date'])
feature_df['review_date'].max()

def rev_func(num_tags):
    num_tags = num_tags[2:-2]
    res = num_tags.strip().split(' \', \' ')
    return res

df['tags_n'] = df['tags'].apply(rev_func)

df = df.explode('tags_n')

In [543]:
# Используя информацию из столбца с адрессом отел, создадим новый признак - страну.
# Кодируем данный признак, добавив в наш датафрейм с признаками для обучения модели.

def get_country(address):
    country = address.split(' ')[-2:]
    country = str(country).replace(',', '').replace("['", "").replace("']", "").replace("'", "")
    if country == 'United Kingdom':
        return 'United Kingdom'
    if country == 'Paris France':
        return 'France'
    if country == 'Amsterdam Netherlands':
        return 'Netherlands'
    if country == 'Milan Italy':
        return 'Italy'
    if country == 'Vienna Austria':
        return 'Austria'
    if country == 'Barcelona Spain':
        return 'Spain'
    else:
        return 'other'
    
df['country'] = df['hotel_address'].apply(get_country)

import category_encoders as ce
encoder = ce.OneHotEncoder(cols=['country'], use_cat_names=True)
type_bin = encoder.fit_transform(df['country'])
feature_df = pd.concat([feature_df, type_bin], axis=1)

# url = 'https://www.belastingdienst.nl/wps/wcm/connect/bldcontenten/belastingdienst/individuals/tax-regulations/tax_treaties/list-of-eu-countries/'
# response = requests.get(url)
# page = BeautifulSoup(response.text, 'html.parser')
# eu_countries = page.find('article').find_all('li')
# eu_countries = list(eu_countries)

# for i in range(len(eu_countries)):
#     eu_countries[i] = (str(eu_countries[i])).replace('<li>','').replace('</li>', '').replace('\xa0', ' ').replace('*', '').replace("'", "")
# eu_countries

# Сначала хотела подтянуть из сети список со странами Европы, чтобы создать признак, 
# но оказалось униклаьных значений совсем немного - в итоге релаизовала идею через функцию.


In [544]:
# Из признака даты получим два новых признака - год и месяц. Так же создадаим признак сезона.


feature_df['year'] = feature_df['review_date'].dt.year
feature_df['month'] = feature_df['review_date'].dt.month


def get_season(data):
    if data in [12, 1, 2]:
        return 'winter'
    if data in [3, 4, 5]:
        return 'spring'
    if data in [6, 7, 8]:
        return 'summer'
    if data in [9, 10, 11]:
        return 'autumn'
    
df['season'] = feature_df['month'].apply(get_season)
season_encoder = ce.OneHotEncoder(cols=['season'], use_cat_names=True) 
season_bin = season_encoder.fit_transform(df['season'])
feature_df = pd.concat([feature_df, season_bin], axis=1)


In [545]:
# Хотела добавить такой признак как национальность ревьюера, но забегая чуть дальше, он оказался не столь информативным. 
# А еще, вероятно, не оччень политкорректным :) Поэтому строки ниже закомментированы.

# df['reviewer_nationality'].unique()

# bin_encoder = ce.BinaryEncoder(cols=['reviewer_nationality']) 
# type_bin = bin_encoder.fit_transform(df['reviewer_nationality'])
# feature_df = pd.concat([feature_df, type_bin], axis=1)

In [546]:
# Преобразуем признак 'days_since_review', оставив только число.

feature_df['days_since_review'] = df['days_since_review'].apply(lambda x: str(x.split(' ')[:1]).replace("'", '').replace("'", '').replace('[', '').replace(']', ''))
feature_df['days_since_review'] = feature_df['days_since_review'].astype('float64')
                                                         

In [547]:
# Скопируем в датасет с признаками необходимые для обучения колонки.

feature_df['additional_number_of_scoring'] = df['additional_number_of_scoring']
feature_df['average_score'] = round(df['average_score'])
feature_df['review_total_negative_word_counts'] = df['review_total_negative_word_counts']
feature_df['total_number_of_reviews'] = df['total_number_of_reviews']
feature_df['review_total_positive_word_counts'] = df['review_total_positive_word_counts']
feature_df['total_number_of_reviews_reviewer_has_given'] = df['total_number_of_reviews_reviewer_has_given']
feature_df['reviewer_score'] = round(df['reviewer_score'])

In [548]:
# Я так же пробовала создать признак, который указывал бы на том, расположен ли отель в столице страны или нет. 
# Но как выяснилось, он так же имеет слишком слабую взаимосвязь с целевым признаком. 

# def if_capital(country, address):
#     if country == 'United Kingdom':
#         if 'London' in address:
#             return 'yes'
#         else:
#             return 'no'
#     if country == 'France':
#         if 'Paris' in address:
#             return 'yes'
#         else:
#             return 'no'
#     if country == 'Netherlands':
#         if 'Amsterdam' in address:
#             return 'yes'
#         else:
#             return 'no'
#     if country == 'Italy':
#         if 'Milan' in address:
#             return 'yes'
#         else:
#             return 'no'
#     if country == 'Austria':
#         if 'Vienna' in address:
#             return 'yes'
#         else:
#             return 'no'
#     if country == 'Spain':
#         if 'Barcelona' in address:
#             return 'yes'
#         else:
#             return 'no'

# df['if_capital'] = df.apply(lambda row: if_capital(row['country'], row['hotel_address']), axis = 1)

# encodery = ce.OneHotEncoder(cols=['if_capital'], use_cat_names=True) 
# type_biny = encodery.fit_transform(df['if_capital'])
# feature_df = pd.concat([feature_df, type_biny], axis=1)

In [549]:
# df['tags'] = df['tags'].apply(lambda x: x.replace("' ", "'").replace(" '", "'").replace("[", "").replace("]", "").split(','))

# Мне хотелось обработать тэги, а так же вычленить полезную информацию из негативных и положительных отзывов, 
# но к сожалению, я не смогла придумать как.

In [550]:
# Создаю: 1) признак с координатами отеля;   
# 2) признак с координатами центра города (за центр города я брала координаты с Google Maps, ткнув точку в центр города)

def get_coordinate(lat, lng):
    return (float(lat), float(lng))

def capital_coordinate(country):
    if country == 'United Kingdom':
        return (float(51.50936092785646), float(-0.12416095345639586))
    if country == 'France':
        return (float(48.853229), float(2.342797))
    if country == 'Netherlands':
        return (float(52.37176471692262), float(4.895930820099642))
    if country == 'Italy':
        return (float(45.46986), float(9.18499))
    if country == 'Austria':
        return (float(48.20471741160276), float(16.37049992180316))
    if country == 'Spain':
        return (float(41.395704279217775), float(2.172020436193314))
        


df['hotel_coordinate'] = df.apply(lambda row: get_coordinate(row['lat'], row['lng']), axis = 1)
df['center_coordinate'] = df['country'].apply(capital_coordinate)


In [551]:
# Создаю признак с расстоянием от центра города до отеля 

from geopy import distance

def dist_from_cent(hotel_coordinate, center_coordinate):
    return geodesic(hotel_coordinate, center_coordinate).kilometers

feature_df['dist_from_cent_km'] = round(df.apply(lambda row: dist_from_cent(row['hotel_coordinate'], row['center_coordinate']),  axis = 1))

In [552]:
# Добавляю признак "город"

def city(country):
    if country == 'United Kingdom':
        return 'London'
    if country == 'France':
        return 'Paris'
    if country == 'Netherlands':
        return 'Amsterdam'
    if country == 'Italy':
        return 'Milan'
    if country == 'Austria':
        return 'Vienna'
    if country == 'Spain':
        return 'Barcelona'

df['city'] = df['country'].apply(city)

# Я так же пробовала кодировать данный признак и смотреть на матрицу корреляции, не заметив связи между целевым признаком и городом, 
# я решила оставить его просто в датайфрейме для размышления, не передавая его модели.

In [559]:
feature_df = feature_df.drop('review_date', axis = 1)

,country_United Kingdom,country_France,country_Netherlands,country_Italy,country_Austria,country_Spain,year,month,season_winter,season_autumn,season_spring,season_summer,days_since_review,additional_number_of_scoring,average_score,review_total_negative_word_counts,total_number_of_reviews,review_total_positive_word_counts,total_number_of_reviews_reviewer_has_given,reviewer_score,dist_from_cent_km
country_United Kingdom,1.000000,-0.371575,-0.365294,-0.286029,-0.280528,-0.374451,0.003347,-0.023546,0.058668,-0.015748,0.010478,-0.050575,0.007904,0.439220,-0.065162,0.022929,0.106828,-0.061677,-0.085910,-0.041237,0.317148
country_France,-0.371575,1.000000,-0.128601,-0.100696,-0.098759,-0.131824,-0.016302,0.013279,-0.027124,0.015953,-0.006964,0.017219,0.012892,-0.254451,0.013977,-0.021062,-0.229490,0.018859,0.021325,0.003481,-0.062763
country_Netherlands,-0.365294,-0.128601,1.000000,-0.098994,-0.097090,-0.129596,0.024582,-0.015408,0.004502,-0.012049,0.007147,0.000244,-0.022242,-0.007134,-0.013966,0.011861,0.102534,0.029998,-0.010569,0.011523,-0.086738
country_Italy,-0.286029,-0.100696,-0.098994,1.000000,-0.076022,-0.101475,0.003569,0.010952,-0.035966,0.007888,-0.000446,0.026883,-0.009566,-0.130306,-0.010279,-0.010116,0.086120,0.003387,0.074298,-0.007768,-0.067954
country_Austria,-0.280528,-0.098759,-0.097090,-0.076022,1.000000,-0.099523,-0.017003,0.021904,0.006708,0.002429,-0.018948,0.009744,0.009642,-0.163865,0.059608,-0.016369,-0.050177,0.014669,0.076172,0.027733,-0.149516
country_Spain,-0.374451,-0.131824,-0.129596,-0.101475,-0.099523,1.000000,-0.002548,0.012399,-0.045007,0.012175,-0.001043,0.031927,-0.003206,-0.188308,0.062212,-0.005267,-0.067949,0.033330,0.002802,0.033630,-0.172534
year,0.003347,-0.016302,0.024582,0.003569,-0.017003,-0.002548,1.000000,-0.580706,0.065027,-0.477335,0.318568,0.082155,-0.917104,-0.004841,0.008492,0.037812,-0.003783,0.045923,-0.036841,0.010576,0.019049
month,-0.023546,0.013279,-0.015408,0.010952,0.021904,0.012399,-0.580706,1.000000,-0.251886,0.574190,-0.424747,0.105268,0.209869,-0.013876,-0.005601,-0.009677,-0.005316,-0.026553,0.016360,-0.031554,-0.008022
season_winter,0.058668,-0.027124,0.004502,-0.035966,0.006708,-0.045007,0.065027,-0.251886,1.000000,-0.306422,-0.320428,-0.342153,0.043714,0.030555,0.006756,-0.011646,0.010384,-0.010849,0.006670,0.028464,0.000853
season_autumn,-0.015748,0.015953,-0.012049,0.007888,0.002429,0.012175,-0.477335,0.574190,-0.306422,1.000000,-0.323305,-0.345225,0.292933,-0.009361,-0.008344,-0.011705,-0.004149,-0.033063,0.015108,-0.033140,-0.008497


In [561]:
feature_df

,country_United Kingdom,country_France,country_Netherlands,country_Italy,country_Austria,country_Spain,year,month,season_winter,season_autumn,season_spring,season_summer,days_since_review,additional_number_of_scoring,average_score,review_total_negative_word_counts,total_number_of_reviews,review_total_positive_word_counts,total_number_of_reviews_reviewer_has_given,reviewer_score,dist_from_cent_km
0,1,0,0,0,0,0,2016,2,1,0,0,0,531.0,581,8.0,3,1994,4,7,10.0,1.0
0,1,0,0,0,0,0,2016,2,1,0,0,0,531.0,581,8.0,3,1994,4,7,10.0,1.0
0,1,0,0,0,0,0,2016,2,1,0,0,0,531.0,581,8.0,3,1994,4,7,10.0,1.0
0,1,0,0,0,0,0,2016,2,1,0,0,0,531.0,581,8.0,3,1994,4,7,10.0,1.0
0,1,0,0,0,0,0,2016,2,1,0,0,0,531.0,581,8.0,3,1994,4,7,10.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
386801,1,0,0,0,0,0,2017,5,0,0,1,0,74.0,365,8.0,0,1567,6,28,9.0,3.0
386802,1,0,0,0,0,0,2016,8,0,0,0,1,363.0,222,9.0,20,1209,20,2,9.0,5.0
386802,1,0,0,0,0,0,2016,8,0,0,0,1,363.0,222,9.0,20,1209,20,2,9.0,5.0
386802,1,0,0,0,0,0,2016,8,0,0,0,1,363.0,222,9.0,20,1209,20,2,9.0,5.0


In [566]:
# Чтото пошло не так с попыткой нормализовать признаки:

col_names = ['days_since_review', 'additional_number_of_scoring', 'total_number_of_reviews', 'total_number_of_reviews_reviewer_has_given']

from sklearn import preprocessing
r_scaler = preprocessing.RobustScaler()

r_scaler = preprocessing.RobustScaler()
df_r = r_scaler.fit_transform(feature_df)

df_r = pd.DataFrame(df_r, columns=col_names)

fig, (ax1) = plt.subplots(ncols=1, figsize=(10, 8))
ax1.set_title('Распределения после RobustScaler')

sns.kdeplot(df_r)


ValueError: Shape of passed values is (1756505, 21), indices imply (1756505, 4)

In [556]:
X = feature_df.drop(['reviewer_score'], axis = 1)  
y = feature_df['reviewer_score'] 

from sklearn.model_selection import train_test_split  

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42)

# Импортируем необходимые библиотеки:  
from sklearn.ensemble import RandomForestRegressor # инструмент для создания и обучения модели  
from sklearn import metrics # инструменты для оценки точности модели  
  
# Создаём модель  
regr = RandomForestRegressor(n_estimators=100)  
      
# Обучаем модель на тестовом наборе данных  
regr.fit(X_train, y_train)  
      
# Используем обученную модель для предсказания рейтинга отелей в тестовой выборке.  
# Предсказанные значения записываем в переменную y_pred  
y_pred = regr.predict(X_test)  

print('MAPE:', metrics.mean_absolute_percentage_error(y_test, y_pred))



В итоге, у меня получилось даже большее значение, чем при решении взять средний score по отелям.
Очевидно, мне очень нужна помощь и по возможности развернутые комментарии на тему того, что я делала не так, как сделать так...

У меня еще остались несоклько вопросов:

1) Я не очень поняла на Kaggle зачем нам данные test. Как обучать модель на train, а тестировать на других данных? 
Как все это большое решение (преобразование признаков) передать в submission для того, чтобы сделать сабмит?
2) Насоколько нормально для модели такое количество призщнаков? что вообще лучше - больше или меньше? 
Я пыталась сохранять в ноутбуке все что я делала - удаляла и добавляла признаки, но в итоге получилась путанница. И я не сделала никаких логических выводов.
3) по DataFrame.corr() кажется, что на нашу целевую переменную влияют в основном три признака: average_score; 
review_total_negative_word_counts; review_total_positive_word_counts.
Неадекватно ведь брать три признака для модели?..
А адекватно ли брать признаки, которые не особо взаимосвязаны с целеврой переменной, которую необходимо предсказать?
4) про нормализацию: я понимаю, что есть бинарные признаки, значения в этих столбцах равно нулю или единице. Понимаю, что есть признаки с иным порядком чисел: например значения в столбце с days_since_review будут сильно отличаться от 0 и 1. Соотвтетсвенно, модели нужна нормальзация признаков? Всех? Ведь тогда признак года, по идее, тоже сильно выбивается из других значений признаков. С моей попыткой чтото пошло не так.
5) На Kaggle у меня вылезает ошибка при попытке отправить сабмишен (ноутбук сохранился, но рядом сним красный треугольник, внутри которого восклицательный знак, написано error :( ). Хотя в output сохранены данные. Вероятно, я не понимаючто конкретно надо отправлять в сабмишен.

Я честно перечитывала и смотрела видео несколько раз, но не разобралась...
Заранее спасибо за помощь :)